In [ ]:
# リスト 5.2.5
# 学習済みWord2Vecデータのロード

import gensim

model = gensim.models.Word2Vec.load("ja.bin")
print(model)

In [ ]:
# リスト 5.2.6
# 学習済みWord2Vecの挙動を調べる

# 「世間」の特徴量ベクトルを調べる
print("「世間」の特徴量ベクトル")
print(model.wv["世間"])

# 「世間」の類似語を調べる
print()
print("「世間」の類似語")
for item, value in model.wv.most_similar("世間"):
    print(item, value)